# Daq Setup

In [1]:
import pynq

import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..')))

from Biquad.Biquad_Daq import Biquad_Daq

from Utils.PlotTools import PlottingTools as pt

from Waveforms.Waveform import Waveform
from Waveforms.Gated import Gated
from Waveforms.Filterred import Filterred

import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual, HBox
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
daq = Biquad_Daq(None, None, 4, 2**10)

# daq.setA(1)
# daq.setB(0)
# daq.setP(0)
# daq.setTheta(np.pi)
# daq.setM(8)

Turning on SYNC
Turning off SYNC


# Sim Setup

In [2]:
from Biquad.SimBiquad import SimBiquad

# Patricks Biquad

In [3]:
from scipy.special import eval_chebyu
import scipy.signal as signal

def fir_biquad( Data , A, B ):
    result = []

    result.append(A*Data[0])
    result.append(A*Data[1] + B*Data[0])
    
    for n in range(2, len(Data)):
        result.append(A*Data[n] + B*Data[n-1] + A*Data[n-2])
            
    return np.array(result)
    

def iir_biquad( ins , samp_per_clock, mag, angle, ics = None ):
    
    if ics is None:
        # Debugging
        # print("No initial conditions!")
        ics = np.zeros(samp_per_clock*3)
        ics = ics.reshape(3,-1)

    # Generate the FIR coefficients for the "f" term (constant for sample 0)
    # These go from i=0 to i=samp_per_clock-2
    f_fir = [0]*(samp_per_clock-2)
    for i in range(0, samp_per_clock-2):
        f_fir[i] = pow(mag, i)*eval_chebyu(i,np.cos(angle) )
    # And the same for the "g" term (constant for sample 1).
    # This is written a bit differently in the document b/c of the block representation.
    # But remember ANY FIR costs only num_tap DSPs period for each sample.
    # This is just the same as the other FIR but with 1 add'l tap
    g_fir = [0]*(samp_per_clock-1)
    for i in range(0, samp_per_clock-1):
        g_fir[i] = pow(mag, i)*eval_chebyu(i,np.cos(angle) )
    # Note f_fir[0]/g_fir[0] are going to both be 1

    # Debugging.
    # print("Magnitude:", mag, "Angle:", angle)
    # print("f/g FIRs are calculated only for sample 0 and sample 1 respectively.")
    # print("f FIR:", f_fir)
    # print("g FIR:", g_fir)

    # Expand the inputs with the initial conditions
    newins = np.concatenate( (ics[0],ins) )
    
    # Run the FIRs
    f = signal.lfilter( f_fir, [1], newins )
    g = signal.lfilter( g_fir, [1], newins )
    # Now we decimate f/g by 8, because we only want the 0th and 1st samples out of 8 for each.
    f = f.reshape(-1, samp_per_clock).transpose()[0]
    g = g.reshape(-1, samp_per_clock).transpose()[1]
    
    # n.b. f[0]/g[0] are initial conditions
    
    # f[2] (real sample 8) is calculated from 8, 7, 6, 5, 4, 3, 2.
    # g[2] (real sample 9) is calculated from 9, 8, 7, 6, 5, 4, 3, 2.

    # Now we need to compute the F and G functions, which *again* are just FIRs,
    # however they're cross-linked, so it's a little trickier.
    # We split it into
    # F = (fir of f) + G_coeff*g(previous clock)
    # G = (fir of g) + F_coeff*f(previous clock)
    F_fir = [ 1.0, -1*pow(mag, samp_per_clock)*eval_chebyu(samp_per_clock-2, np.cos(angle)) ]
    G_fir = [ 1.0, pow(mag, samp_per_clock)*eval_chebyu(samp_per_clock, np.cos(angle)) ]

    # Crosslink coefficients. 
    Coeff_g_in_F = pow(mag, samp_per_clock-1)*eval_chebyu(samp_per_clock-1, np.cos(angle))
    Coeff_f_in_G = pow(mag, samp_per_clock+1)*eval_chebyu(samp_per_clock-1, np.cos(angle))
    
    # Debugging
    # print("F/G FIRs operate on f/g inputs respectively")
    # print("F FIR:", F_fir, "+g*", Coeff_g_in_F)
    # print("G FIR:", G_fir, "-f*", Coeff_f_in_G)
    # print()
    # print("As full FIRs calculated only for sample 0 and 1 respectively:")
    # print("F = f + (fz^-", samp_per_clock, ")+",Coeff_g_in_F,"*(gz^-", samp_per_clock-1, ")",sep='')
    # print("G = g + (gz^-", samp_per_clock, ")-",Coeff_f_in_G,"*(fz^-", samp_per_clock+1, ")",sep='')
        
    # Filter them
    F = signal.lfilter( F_fir, [1], f )
    G = signal.lfilter( G_fir, [1], g )

    # Now we need to feed the f/g paths into the opposite filter
    # F[0]/G[0] are going to be dropped anyway.
    F[1:] += Coeff_g_in_F*g[0:-1]
    G[1:] -= Coeff_f_in_G*f[0:-1]    

    # drop the initial conditions
    F = F[1:]
    G = G[1:]
    
    # Now reshape our outputs.
    arr = ins.reshape(-1, samp_per_clock).transpose()
    # arr[0] is now every 0th sample (e.g. for samp_per_clock = 8, it's 0, 8, 16, 24, etc.)
    # arr[1] is now every 1st sample (e.g. for samp_per_clock = 8, it's 1, 9, 17, 25, etc.)

    # IIR parameters. See the 'update step' in paper.
    C = np.zeros(4)
    C[0] = pow(mag, 2*samp_per_clock)*(pow(eval_chebyu(samp_per_clock-2, np.cos(angle)), 2) -
                                       pow(eval_chebyu(samp_per_clock-1, np.cos(angle)), 2))

    C[1] = pow(mag, 2*samp_per_clock-1)*((eval_chebyu(samp_per_clock-1, np.cos(angle)))*
                                         (eval_chebyu(samp_per_clock,np.cos(angle)) -
                                          eval_chebyu(samp_per_clock-2, np.cos(angle))))

    C[2] = pow(mag, 2*samp_per_clock+1)*((eval_chebyu(samp_per_clock-1, np.cos(angle)))*
                                         (eval_chebyu(samp_per_clock-2, np.cos(angle))-
                                          eval_chebyu(samp_per_clock, np.cos(angle))))

    C[3] = pow(mag, 2*samp_per_clock)*(pow(eval_chebyu(samp_per_clock, np.cos(angle)), 2) -
                                       pow(eval_chebyu(samp_per_clock-1, np.cos(angle)), 2))

    # Debugging
    # print("Update step (matrix) coefficients:", C)
    # print("As an IIR:")
    # print("y[0] =", C[1], "*z^-", samp_per_clock*2-1," + ", C[0], "*z^-", samp_per_clock*2,
    #       "+F[0]",sep='')
    # print("y[1] =", C[3], "*z^-", samp_per_clock*2," + ", C[2], "*z^-", samp_per_clock*2+1,
    #       "+G[1]",sep='')
    # Now compute the IIR.
    # INITIAL CONDITIONS STEP
    y0_0 =  C[0]*ics[1][0] + C[1]*ics[1][1] + F[0]
    y1_0 =  C[2]*ics[1][0] + C[3]*ics[1][1] + G[0]
    y0_1 =  C[0]*ics[2][0] + C[1]*ics[2][1] + F[1]
    y1_1 =  C[2]*ics[2][0] + C[3]*ics[2][1] + G[1]
    for i in range(len(arr[0])):
        if i == 0:
            # Compute from initial conditions.
            arr[0][i] = C[0]*y0_0 + C[1]*y1_0 + F[i]
            arr[1][i] = C[2]*y0_0 + C[3]*y1_0 + G[i]
        elif i==1:
            # Compute from initial conditions
            arr[0][i] = C[0]*y0_1 + C[1]*y1_1 + F[i]
            arr[1][i] = C[2]*y0_1 + C[3]*y1_1 + G[i]
        else:
            # THIS IS THE ONLY RECURSIVE STEP
            arr[0][i] = C[0]*arr[0][i-2] + C[1]*arr[1][i-2] + F[i]
            arr[1][i] = C[2]*arr[0][i-2] + C[3]*arr[1][i-2] + G[i]

        # THIS IS NOT RECURSIVE B/C WE DO NOT TOUCH THE SECOND INDEX
        for j in range(2, samp_per_clock):
            arr[j][i] += 2*mag*np.cos(angle)*arr[j-1][i] - pow(mag, 2)*arr[j-2][i]

    # now transpose arr and flatten
    return arr.transpose().reshape(-1)

def modify_array(arr):
    # Length of the array
    n = len(arr)
    
    # Iterate over the array
    for i in range(n):
        # Check if the index should remain unchanged
        if not ((i % 8 == 0) or (i % 8 == 1)):
            arr[i] = 0
    return arr

## Update the biquad programming

In [4]:
def update_daq(daq, A, B, P, Theta, M):
    daq.setA(A)
    daq.setB(B)
    daq.setP(P)
    daq.setTheta(np.pi*Theta)
    
    daq.set_single_zero_fir()
    daq.set_f_fir(0)
    daq.set_g_fir(0)
    
    return daq
    # daq.sdv.write(0x10, 0)
    # daq.sdv.write(0x14, 0)
    # daq.sdv.write(0x00,1)
    
    # daq.set_F_fir()
    # daq.set_G_fir()
    # daq.set_pole_coef()
    # daq.set_incremental()
    
def replicate_sim(biquad):
    biquad.set_daq_coeffs(daq.get_coeffs())
    biquad.single_zero_fir()
    biquad.first_constants()
    return biquad

def update_sim(A, B, P, Theta, M, biquad):
    biquad.setA(A)
    biquad.setB(B)
    biquad.setP(P)
    biquad.setTheta(np.pi*Theta)
    
    biquad.IIR()
    
    return biquad


def test(data, A, B, mag, angle):
    return modify_array(iir_biquad(fir_biquad(data, A, B), 8, mag, angle))
    # return modify_array(iir_biquad(data, 8, mag, angle))

In [8]:
def update_plot(A, B, P, Theta, M):
    daq = update_daq(daq, A, B, P, Theta, M)
    
    daq.JupyterAcquire()
    
    fig, (axD,axS,axP) = plt.subplots(3, 1, figsize=(25, 20))
    
    daq.waveforms[2].plotWaveform(axD, title="Biquaded ADC224_T0_CH0")


    biquad = SimBiquad(data=daq.adcBuffer[0] >> 4, A=A, B=B, P=P, theta=Theta)    

    biquad = replicate_sim(biquad)
    # biquad = update_sim(A, B, P, Theta, M, biquad)
    
    sim_output = Filterred(biquad.get_decimated1(), 280, 800)
    
    sim_output.plotWaveform(axS, title="Simulated Biquad")
    
    
    diff_output = Filterred(np.array(daq.waveforms[2].waveform)-biquad.get_decimated1()[280:800], 0, None)
    
    diff_output.plotWaveform(axP, title="Daq Biquad minus Sim Biquad")
    
    # other_output = SimFilter(test(daq.adcBuffer[0] >> 4, A, B, P, Theta*np.pi))
    # other_output.plotBiquad(axP, title="DSP python Biquad")
    # axP.plot(test(daq.adcBuffer[0] >> 4, A, B, P, Theta))
    
    # print(daq.printCoeffs())
    
    print(daq.waveforms[2].waveform[0:2])
    print(sim_output.waveform[0:2])
    
    plt.show()
    
A_slider = widgets.IntSlider(min=-8, max=7, step=1, value=1, description='A:', continuous_update=False)
B_slider = widgets.IntSlider(min=-8, max=7, step=1, value=1, description='B:', continuous_update=False)
P_slider = widgets.FloatSlider(min=0, max=2, step=0.05, value=0.2, description='P:', continuous_update=False)
Theta_slider = widgets.FloatSlider(min=0, max=2, step=0.1, value=0.5, description='Theta:', continuous_update=False)
M_slider = widgets.IntSlider(min=0, max=30, step=1, value=8, description='M:', continuous_update=False)
    
# Create interactive plot
plot_output = widgets.Output()
    
def update(change):
    with plot_output:
        plot_output.clear_output(wait=True)
        update_plot(A_slider.value, B_slider.value, P_slider.value, Theta_slider.value, M_slider.value)
    
A_slider.observe(update, names='value')
B_slider.observe(update, names='value')
P_slider.observe(update, names='value')
Theta_slider.observe(update, names='value')
M_slider.observe(update, names='value')

# Layout
slider_box = widgets.HBox([A_slider, B_slider, P_slider, Theta_slider, M_slider])
ui = widgets.VBox([slider_box, plot_output])

# Display
display(ui)

# Initial plot
update(None)

In [6]:
# def test():
#     orig = daq.adcBuffer[0] >> 4
#     b, a = signal.iirnotch(400, 5, 3000)
#     pole = signal.tf2zpk(b,a)[1][0]
#     mag=np.abs(pole)
#     angle=np.angle(pole)
    
#     mag = 0
#     angle = np.pi
#     return iir_biquad(orig, 8, mag, angle)

# fig, (ax0,ax1) = plt.subplots(2, 1, figsize=(25, 20))

# result = modify_array(test())

# print(result)

# ax0.plot(daq.adcBuffer[0] >> 4)
# ax1.plot(result)